Baseline - ARIMA

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import linregress
import os

# Optional for larger text and better visuals
sns.set(style="whitegrid", context="talk")

In [12]:
# Load the sales data (tab-delimited)
sales = pd.read_csv("datasets/sell-in.txt", sep="\t", dtype={"periodo": str})

# Load the stocks data (tab-delimited)
stocks = pd.read_csv("datasets/tb_stocks.txt", sep="\t", dtype={"periodo": str})

# Load the product information data (tab-delimited)
product_info = pd.read_csv("datasets/tb_productos.txt", sep="\t")

#carga productos a predecir
product_predict = pd.read_csv("datasets/product_id_apredecir201912.txt", sep="\t",header=0)

In [13]:
# Asegurar que 'periodo' sea string
sales['periodo'] = sales['periodo'].astype(str)



In [14]:
# Paso 1: agrupar por producto y periodo, sumando tn por mes
tn_por_producto_y_periodo = (
    sales
    .groupby(['product_id', 'periodo'])['tn']
    .sum()
    .reset_index()
)


In [15]:
tn_por_producto_y_periodo.head()

,product_id,periodo,tn
0,20001,201701,934.77222
1,20001,201702,798.01620
2,20001,201703,1303.35771
3,20001,201704,1069.96130
4,20001,201705,1502.20132


In [16]:
# Filtrar tn_por_producto_y_periodo
tn_por_producto_y_periodo_a_predecir= tn_por_producto_y_periodo[tn_por_producto_y_periodo['product_id'].isin(product_predict['product_id'])]

# Mostrar el resultado
print(tn_por_producto_y_periodo_a_predecir)

       product_id periodo          tn
0           20001  201701   934.77222
1           20001  201702   798.01620
2           20001  201703  1303.35771
3           20001  201704  1069.96130
4           20001  201705  1502.20132
...           ...     ...         ...
31206       21276  201908     0.01265
31207       21276  201909     0.01856
31208       21276  201910     0.02079
31209       21276  201911     0.03341
31210       21276  201912     0.00892

[22349 rows x 3 columns]


In [ ]:
#tn_por_producto_y_periodo_a_predecir.to_csv("datasets/tn_por_producto_y_periodo_a_predecir.csv", index=False, encoding="utf-8")

In [18]:
sales.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452


In [20]:
from statsmodels.tsa.arima.model import ARIMA
from tqdm import tqdm

# Limpiar columnas
sales.columns = sales.columns.str.strip()
product_predict.columns = product_predict.columns.str.strip()

In [21]:
# Convertir periodo a string y luego a datetime con formato año-mes
sales['periodo'] = sales['periodo'].astype(str)
sales['periodo'] = pd.to_datetime(sales['periodo'], format='%Y%m')

In [23]:
# Filtrar productos a predecir
sales = sales[sales['product_id'].isin(product_predict['product_id'])]

In [25]:
# Agrupar por producto y periodo
ventas_agrupadas = (
    sales
    .groupby(['product_id', 'periodo'])['tn']
    .sum()
    .reset_index()
    .sort_values(['product_id', 'periodo'])
)

In [ ]:
#pip install statsforecast


   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 2.8/2.8 MB 23.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/30.3 MB ? eta -:--:--
   ---- ----------------------------------- 3.7/30.3 MB 19.8 MB/s eta 0:00:02
   --------- ------------------------------ 7.1/30.3 MB 17.4 MB/s eta 0:00:02
   ----------- ---------------------------- 8.4/30.3 MB 13.7 MB/s eta 0:00:02
   --------------- ------------------------ 11.8/30.3 MB 16.0 MB/s eta 0:00:02
   -------------------- ------------------- 15.5/30.3 MB 15.2 MB/s eta 0:00:01
   -------------------------- ------------- 20.2/30.3 MB 16.3 MB/s eta 0:00:01
   -------------------------------- ------- 24.6/30.3 MB 17.3 MB/s eta 0:00:01
   -------------------------------------- - 28.8/30.3 MB 17.7 MB/s eta 0:00:01
   ---------------------------------------- 30.3/30.3 MB 17.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [35]:
from statsforecast.models import AutoARIMA
from tqdm import tqdm

# Lista de resultados
resultados = []

# Loop con AutoARIMA de statsforecast
for producto in tqdm(product_predict['product_id']):
    serie = ventas_agrupadas[ventas_agrupadas['product_id'] == producto]
    serie = serie.set_index('periodo')
    ts = serie['tn']

    # Validaciones mínimas
    if ts.isnull().any() or len(ts) < 12 or ts.sum() == 0 or ts.nunique() == 1:
        forecast = 0
    else:
        try:
            modelo = AutoARIMA(season_length=12)  # Define estacionalidad de 12 meses
            modelo.fit(ts)
            forecast = modelo.predict(1)[0]  # Predicción a 1 periodo futuro
            forecast = max(forecast, 0)
        except:
            forecast = 0

    resultados.append({'product_id': producto, 'tn': forecast})



  0%|          | 0/780 [00:00<?, ?it/s]c:\Users\paola\anaconda3\envs\ldi2\Lib\site-packages\statsforecast\arima.py:1123: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return np.all(x[0] == x)
c:\Users\paola\anaconda3\envs\ldi2\Lib\site-packages\statsforecast\arima.py:1123: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return np.all(x[0] == x)
  0%|          | 1/780 [00:00<08:34,  1.52it/s]c:\Users\paola\anaconda3\envs\ldi2\Lib\site-packages\statsforecast\arima.py:1123: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consis

In [46]:
resultados

[{'product_id': 20001, 'tn': 0},
 {'product_id': 20002, 'tn': 0},
 {'product_id': 20003, 'tn': 0},
 {'product_id': 20004, 'tn': 0},
 {'product_id': 20005, 'tn': 0},
 {'product_id': 20006, 'tn': 0},
 {'product_id': 20007, 'tn': 0},
 {'product_id': 20008, 'tn': 0},
 {'product_id': 20009, 'tn': 0},
 {'product_id': 20010, 'tn': 0},
 {'product_id': 20011, 'tn': 0},
 {'product_id': 20012, 'tn': 0},
 {'product_id': 20013, 'tn': 0},
 {'product_id': 20014, 'tn': 0},
 {'product_id': 20015, 'tn': 0},
 {'product_id': 20016, 'tn': 0},
 {'product_id': 20017, 'tn': 0},
 {'product_id': 20018, 'tn': 0},
 {'product_id': 20019, 'tn': 0},
 {'product_id': 20020, 'tn': 0},
 {'product_id': 20021, 'tn': 0},
 {'product_id': 20022, 'tn': 0},
 {'product_id': 20023, 'tn': 0},
 {'product_id': 20024, 'tn': 0},
 {'product_id': 20025, 'tn': 0},
 {'product_id': 20026, 'tn': 0},
 {'product_id': 20027, 'tn': 0},
 {'product_id': 20028, 'tn': 0},
 {'product_id': 20029, 'tn': 0},
 {'product_id': 20030, 'tn': 0},
 {'product

In [ ]:
# Guardar resultados
df_predicciones = pd.DataFrame(resultados)
df_predicciones.to_csv("data/pred_autoarima2.csv", index=False, encoding="utf-8")
print("✅ Archivo submission_arima.csv generado correctamente.")

✅ Archivo submission_arima.csv generado correctamente.


In [10]:
submission.to_csv("data/pred_trivial_media12.csv", index=False, encoding="utf-8")

In [ ]:
#!pip install pmdarima

   ---------------------------------------- 0.0/625.1 kB ? eta -:--:--
   --------------------------------------- 625.1/625.1 kB 11.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 25.9 MB/s eta 0:00:00


In [49]:
from pmdarima import auto_arima
from tqdm import tqdm
# Lista de resultados
resultados = []

# Loop con auto_arima
for producto in tqdm(product_predict['product_id']):
    serie = ventas_agrupadas[ventas_agrupadas['product_id'] == producto]
    serie = serie.set_index('periodo')
    ts = serie['tn']

    if ts.isnull().any() or len(ts) < 6 or ts.sum() == 0 or ts.nunique() == 1:
        forecast = 0
    else:
        try:
            modelo = auto_arima(ts,
                                seasonal=True,
                                m=12,  # Asumiendo estacionalidad mensual
                                stepwise=True,
                                suppress_warnings=True,
                                error_action='ignore',
                                max_order=10)
            forecast = modelo.predict(n_periods=1)[0]
            forecast = max(forecast, 0)
        except:
            forecast = 0

    resultados.append({'product_id': producto, 'tn': forecast})

  0%|          | 0/780 [00:00<?, ?it/s]C:\Users\paola\AppData\Local\Temp\ipykernel_24936\3054401920.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  forecast = modelo.predict(n_periods=1)[0]
  0%|          | 1/780 [00:03<48:48,  3.76s/it]C:\Users\paola\AppData\Local\Temp\ipykernel_24936\3054401920.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  forecast = modelo.predict(n_periods=1)[0]
  0%|          | 2/780 [00:06<41:48,  3.22s/it]C:\Users\paola\AppData\Local\Temp\ipykernel_24936\3054401920.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys w

In [61]:
resultados


[{'product_id': 20001, 'tn': 1488.5546308964779},
 {'product_id': 20002, 'tn': 1607.726836331221},
 {'product_id': 20003, 'tn': 966.6852557631702},
 {'product_id': 20004, 'tn': 668.7699933423313},
 {'product_id': 20005, 'tn': 559.5337793205729},
 {'product_id': 20006, 'tn': 527.092951112689},
 {'product_id': 20007, 'tn': 367.54762859009514},
 {'product_id': 20008, 'tn': 436.6037885230104},
 {'product_id': 20009, 'tn': 572.4654011546966},
 {'product_id': 20010, 'tn': 427.5222987173892},
 {'product_id': 20011, 'tn': 342.49302949270384},
 {'product_id': 20012, 'tn': 292.63725289813163},
 {'product_id': 20013, 'tn': 317.5090727383872},
 {'product_id': 20014, 'tn': 449.3815097222222},
 {'product_id': 20015, 'tn': 374.76837282712063},
 {'product_id': 20016, 'tn': 283.27567211065224},
 {'product_id': 20017, 'tn': 322.7946536072753},
 {'product_id': 20018, 'tn': 141.63569999999999},
 {'product_id': 20019, 'tn': 347.2231402000648},
 {'product_id': 20020, 'tn': 266.06358},
 {'product_id': 20021,

In [51]:
# Verificar si hay al menos un 0 en tn
tiene_ceros = any(r['tn'] == 0 for r in resultados)

print(f"¿Existen valores tn = 0? {'Sí' if tiene_ceros else 'No'}")


¿Existen valores tn = 0? Sí


In [55]:
#carga productos a predecir
medias12 = pd.read_csv("data/pred_trivial_media12.csv", sep=",",header=0)

In [56]:
medias12

,product_id,tn
0,20001,1454.732720
1,20002,1175.437142
2,20003,784.976407
3,20004,627.215328
4,20005,668.270104
...,...,...
775,21263,0.029993
776,21265,0.089541
777,21266,0.094659
778,21267,0.092835


In [59]:
import pandas as pd

# Unir df_resultados con medias12 usando product_id
df_resultados = df_resultados.merge(medias12[['product_id', 'tn']], on='product_id', how='left', suffixes=('', '_media12'))

# Reemplazar valores 0 en tn con el tn de medias12 cuando corresponda
df_resultados['tn'] = df_resultados.apply(lambda row: row['tn_media12'] if row['tn'] == 0 else row['tn'], axis=1)

# Eliminar la columna auxiliar tn_media12
df_resultados.drop(columns=['tn_media12'], inplace=True)

# Ver los primeros registros
print(df_resultados.head())



   product_id           tn
0       20001  1488.554631
1       20002  1607.726836
2       20003   966.685256
3       20004   668.769993
4       20005   559.533779


In [60]:
df_resultados

,product_id,tn
0,20001,1488.554631
1,20002,1607.726836
2,20003,966.685256
3,20004,668.769993
4,20005,559.533779
...,...,...
775,21263,0.029993
776,21265,0.089541
777,21266,0.094659
778,21267,0.092835


In [58]:
resultados_actualizados

[{'product_id': 20001, 'tn': 1488.5546308964779},
 {'product_id': 20002, 'tn': 1607.726836331221},
 {'product_id': 20003, 'tn': 966.6852557631702},
 {'product_id': 20004, 'tn': 668.7699933423313},
 {'product_id': 20005, 'tn': 559.5337793205729},
 {'product_id': 20006, 'tn': 527.092951112689},
 {'product_id': 20007, 'tn': 367.54762859009514},
 {'product_id': 20008, 'tn': 436.6037885230104},
 {'product_id': 20009, 'tn': 572.4654011546966},
 {'product_id': 20010, 'tn': 427.5222987173892},
 {'product_id': 20011, 'tn': 342.49302949270384},
 {'product_id': 20012, 'tn': 292.63725289813163},
 {'product_id': 20013, 'tn': 317.5090727383872},
 {'product_id': 20014, 'tn': 449.3815097222222},
 {'product_id': 20015, 'tn': 374.76837282712063},
 {'product_id': 20016, 'tn': 283.27567211065224},
 {'product_id': 20017, 'tn': 322.7946536072753},
 {'product_id': 20018, 'tn': 141.63569999999999},
 {'product_id': 20019, 'tn': 347.2231402000648},
 {'product_id': 20020, 'tn': 266.06358},
 {'product_id': 20021,

In [62]:
# Guardar resultados
df_predicciones = pd.DataFrame(resultados_actualizados)
df_predicciones.to_csv("data/pred_autoarima-pmdarima2.csv", index=False, encoding="utf-8")
print("✅ Archivo submission_arima.csv generado correctamente.")

✅ Archivo submission_arima.csv generado correctamente.


In [63]:
df_predicciones

,product_id,tn
0,20001,1488.554631
1,20002,1607.726836
2,20003,966.685256
3,20004,668.769993
4,20005,559.533779
...,...,...
775,21263,0.029993
776,21265,0.089541
777,21266,0.094659
778,21267,0.092835


In [64]:
#carga productos a predecir
medias24 = pd.read_csv("data/pred_trivial_media24.csv", sep=",",header=0)

In [69]:
# Unir ambos DataFrames por product_id
df_promedios = medias12[['product_id', 'tn']].merge(medias24[['product_id', 'tn']], on='product_id', suffixes=('_12', '_24'))

# Calcular el promedio de tn
df_promedios['tn_promedio'] = df_promedios[['tn_12', 'tn_24']].mean(axis=1)

# Crear DataFrame final de resultados con product_id y tn_promedio
resultados = df_promedios[['product_id', 'tn_promedio']]

# Mostrar los primeros registros
print(resultados.head())

   product_id  tn_promedio
0       20001  1467.520290
1       20002  1153.418912
2       20003   807.474489
3       20004   633.851496
4       20005   645.630194


In [70]:
# Renombrar la columna tn_promedio a tn
resultados.rename(columns={'tn_promedio': 'tn'}, inplace=True)

# Ver los primeros registros para confirmar el cambio
print(resultados.head())

   product_id           tn
0       20001  1467.520290
1       20002  1153.418912
2       20003   807.474489
3       20004   633.851496
4       20005   645.630194


C:\Users\paola\AppData\Local\Temp\ipykernel_24936\1513235101.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultados.rename(columns={'tn_promedio': 'tn'}, inplace=True)


In [71]:
resultados

,product_id,tn
0,20001,1467.520290
1,20002,1153.418912
2,20003,807.474489
3,20004,633.851496
4,20005,645.630194
...,...,...
775,21263,0.059613
776,21265,0.089541
777,21266,0.094659
778,21267,0.092835


In [72]:
# Guardar resultados
df_predicciones = pd.DataFrame(resultados)
df_predicciones.to_csv("data/pred_medias_12_24.csv", index=False, encoding="utf-8")
print("✅ Archivo submission_arima.csv generado correctamente.")

✅ Archivo submission_arima.csv generado correctamente.


In [73]:
# Multiplicar todos los valores de tn por 0.97
medias12['tn'] = medias12['tn'] * 0.97

# Ver los primeros registros para confirmar el cambio
print(medias12.head())


   product_id           tn
0       20001  1411.090738
1       20002  1140.174027
2       20003   761.427115
3       20004   608.398868
4       20005   648.222001


In [74]:
df_predicciones = pd.DataFrame(medias12)
df_predicciones.to_csv("data/pred_medias_12_097.csv", index=False, encoding="utf-8")
print("✅ Archivo submission_arima.csv generado correctamente.")

✅ Archivo submission_arima.csv generado correctamente.


In [77]:
#carga productos a predecir
medias12 = pd.read_csv("data/pred_trivial_media12.csv", sep=",",header=0)

In [78]:
# Multiplicar todos los valores de tn por 0.95
medias12['tn'] = medias12['tn'] * 0.95

# Ver los primeros registros para confirmar el cambio
print(medias12.head())

   product_id           tn
0       20001  1381.996084
1       20002  1116.665285
2       20003   745.727587
3       20004   595.854562
4       20005   634.856599


In [79]:
df_predicciones = pd.DataFrame(medias12)
df_predicciones.to_csv("data/pred_medias_12_095.csv", index=False, encoding="utf-8")
print("✅ Archivo submission_arima.csv generado correctamente.")

✅ Archivo submission_arima.csv generado correctamente.


In [89]:
#carga productos a predecir
medias12 = pd.read_csv("data/pred_trivial_media12.csv", sep=",",header=0)

In [84]:

# Multiplicar todos los valores de tn por 0.93
medias12['tn'] = medias12['tn'] * 0.93

In [86]:
df_predicciones = pd.DataFrame(medias12)
df_predicciones.to_csv("data/pred_medias_12_093.csv", index=False, encoding="utf-8")
print("✅ Archivo submission_arima.csv generado correctamente.")

✅ Archivo submission_arima.csv generado correctamente.


In [87]:
#carga productos a predecir
medias12 = pd.read_csv("data/pred_trivial_media12.csv", sep=",",header=0)

In [88]:
# Multiplicar todos los valores de tn por 0.93
medias12['tn'] = medias12['tn'] * 0.89
df_predicciones = pd.DataFrame(medias12)
df_predicciones.to_csv("data/pred_medias_12_089.csv", index=False, encoding="utf-8")
print("✅ Archivo submission_arima.csv generado correctamente.")

✅ Archivo submission_arima.csv generado correctamente.


In [90]:
# Multiplicar todos los valores de tn por 0.93
medias12['tn'] = medias12['tn'] * 0.91
df_predicciones = pd.DataFrame(medias12)
df_predicciones.to_csv("data/pred_medias_12_091.csv", index=False, encoding="utf-8")
print("✅ Archivo submission_arima.csv generado correctamente.")

✅ Archivo submission_arima.csv generado correctamente.
